In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader, random_split
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import resnet34

from PIL import Image
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

from PIL import Image, ImageChops
import math

In [0]:
if not os.path.isdir('./GTEA61'):
    !git clone https://github.com/MauriVass/GTEA61

Cloning into 'GTEA61'...
remote: Enumerating objects: 9639, done.
remote: Counting objects: 100% (9639/9639), done.
remote: Compressing objects: 100% (9566/9566), done.
remote: Total 93346 (delta 25), reused 9577 (delta 8), pack-reused 83707
Receiving objects: 100% (93346/93346), 4.21 GiB | 34.74 MiB/s, done.
Resolving deltas: 100% (1493/1493), done.
Checking out files: 100% (90815/90815), done.


In [30]:
gtea_root = "./GTEA61"
!rm -rf /content/entropies

base_dir_entropies = "/content/entropies"
os.makedirs(base_dir_entropies)

for fst_level_folder in os.listdir(gtea_root):
  if not fst_level_folder.startswith(".") and fst_level_folder != "processed_frames2":
    input_dir = gtea_root + "/" + fst_level_folder
    output_dir = base_dir_entropies + "/" + input_dir
    os.makedirs(output_dir)
    
    for snd_level_folder in os.listdir(input_dir):
      input_dir = gtea_root + "/" + fst_level_folder + "/" + snd_level_folder
      output_dir = base_dir_entropies + "/" + input_dir
      os.makedirs(output_dir)
      
      for trd_level_folder in os.listdir(input_dir):
        input_dir = gtea_root + "/" + fst_level_folder + "/" + snd_level_folder + "/" + trd_level_folder
        output_dir = base_dir_entropies + "/" + input_dir
        os.makedirs(output_dir)

        for vid in os.listdir(input_dir):
          input_dir = gtea_root + "/" + fst_level_folder + "/" + snd_level_folder + "/" + trd_level_folder + "/" + vid
          output_dir = base_dir_entropies + "/" + input_dir
          os.makedirs(output_dir) 

          images_entropy = calculate_images_entropy(input_dir)
          write_file(output_dir, images_entropy)

Writing file /content/entropies/./GTEA61/flow_y_processed/S1/take_honey/1/entropies.txt
Writing file /content/entropies/./GTEA61/flow_y_processed/S1/take_honey/2/entropies.txt
Writing file /content/entropies/./GTEA61/flow_y_processed/S1/take_honey/3/entropies.txt
Writing file /content/entropies/./GTEA61/flow_y_processed/S1/take_spoon/1/entropies.txt
Writing file /content/entropies/./GTEA61/flow_y_processed/S1/take_spoon/5/entropies.txt
Writing file /content/entropies/./GTEA61/flow_y_processed/S1/take_spoon/4/entropies.txt
Writing file /content/entropies/./GTEA61/flow_y_processed/S1/take_spoon/2/entropies.txt
Writing file /content/entropies/./GTEA61/flow_y_processed/S1/take_spoon/3/entropies.txt
Writing file /content/entropies/./GTEA61/flow_y_processed/S1/pour_water,cup/1/entropies.txt
Writing file /content/entropies/./GTEA61/flow_y_processed/S1/pour_water,cup/2/entropies.txt
Writing file /content/entropies/./GTEA61/flow_y_processed/S1/pour_water,cup/3/entropies.txt
Writing file /conten

In [31]:
# Download zip

!zip -r /content/entropies.zip /content/entropies
from google.colab import files
files.download('/content/entropies.zip') 

  adding: content/entropies/ (stored 0%)
  adding: content/entropies/GTEA61/ (stored 0%)
  adding: content/entropies/GTEA61/flow_y_processed/ (stored 0%)
  adding: content/entropies/GTEA61/flow_y_processed/S1/ (stored 0%)
  adding: content/entropies/GTEA61/flow_y_processed/S1/take_honey/ (stored 0%)
  adding: content/entropies/GTEA61/flow_y_processed/S1/take_honey/1/ (stored 0%)
  adding: content/entropies/GTEA61/flow_y_processed/S1/take_honey/1/entropies.txt (deflated 84%)
  adding: content/entropies/GTEA61/flow_y_processed/S1/take_honey/2/ (stored 0%)
  adding: content/entropies/GTEA61/flow_y_processed/S1/take_honey/2/entropies.txt (deflated 85%)
  adding: content/entropies/GTEA61/flow_y_processed/S1/take_honey/3/ (stored 0%)
  adding: content/entropies/GTEA61/flow_y_processed/S1/take_honey/3/entropies.txt (deflated 80%)
  adding: content/entropies/GTEA61/flow_y_processed/S1/take_spoon/ (stored 0%)
  adding: content/entropies/GTEA61/flow_y_processed/S1/take_spoon/1/ (stored 0%)
  add

In [0]:
def image_entropy(img):
    """calculate the entropy of an image"""
    histogram = img.histogram()
    histogram_length = sum(histogram)
    samples_probability = [float(h) / histogram_length for h in histogram]
    return -sum([p * math.log(p, 2) for p in samples_probability if p != 0])

def calculate_images_entropy(input_directory):
    images_entropy = []
    currentFrame = 0

    for current_img_filename in list(sorted(os.listdir(input_directory))):
        if currentFrame > 0:

          prev_filename = os.path.join(input_directory, previous_img_filename)
          curr_filename = os.path.join(input_directory, current_img_filename)

          previous_img = Image.open(prev_filename)
          current_img = Image.open(curr_filename)

          # Compute diff
          img = ImageChops.difference(previous_img, current_img)

          # Compute entropy
          entropy = image_entropy(img)
          images_entropy.append( (currentFrame - 1, currentFrame, prev_filename, curr_filename, entropy) )

        currentFrame += 1
        previous_img_filename = current_img_filename

    return images_entropy

In [0]:
def write_file(output_directory, images_entropy_values):
  print("Writing file " + os.path.join(output_directory, "entropies.txt"))
  with open(os.path.join(output_directory, "entropies.txt"), 'w') as f:
    for row in images_entropy_values:
      f.write(str(row)
        .replace("(", "")
        .replace(")", "") + "\n")